### Importing

In [6]:
import pandas as pd 
import numpy as np

In [7]:
data = pd.read_csv(".\Data\\aviation_accidents in countries - aviation_accidents.csv")

In [3]:
data.shape[0] # number of accidents

21742

In [4]:
data.head()

,Country,date,Air-craft type,registration name/mark,operator,fatilites,location,category
0,Australia,07-Dec-2016,Boeing 737-8FE (WL),VH-VUP,Virgin Australia Airlines,0,Hobart Airpo...,A2
1,Ukraine,19-Feb-1987,Antonov An-26,NaN,Soviet AF,9,near Stadnitsa,A1
2,Iran,19-Feb-2003,Ilyushin Il-76MD,15-2280,Iranian Revolutionary Guard,275,near Kerman,A1
3,Germany,11-May-1990,Transall C-160D,50+39,German AF,10,near Rodenbach,A1
4,Russia,04-Jan-1943,Junkers Ju-52/3m,NaN,German AF,0,Ostrow,A1


In [26]:
nulls_count_precentage={ col_name : (data[col_name].isna().sum(), 
                                str( round( 100 * data[col_name].isna().sum() / data.shape[0] ,2))+" %") 
                                for col_name in data.columns}

In [27]:
nulls_count_precentage

{'Country': (0, '0.0 %'),
 'date': (68, '0.31 %'),
 'Air-craft type': (68, '0.31 %'),
 'registration name/mark': (1453, '6.68 %'),
 'operator': (70, '0.32 %'),
 'fatilites': (3067, '14.11 %'),
 'location': (419, '1.93 %'),
 'category': (71, '0.33 %')}

In [36]:
distinct_count={ col_name : data[col_name].nunique()
                                    for col_name in data.columns}

In [37]:
distinct_count

{'Country': 227,
 'date': 14244,
 'Air-craft type': 3056,
 'registration name/mark': 19855,
 'operator': 5697,
 'fatilites': 359,
 'location': 13401,
 'category': 11}

In [44]:
countries = data['Country'].value_counts()

In [50]:
countries

USA                    4303
Russia                 1410
United Kingdom          836
Canada                  816
India                   696
                       ... 
St. Kitts and Nevis       1
Qatar                     1
Dominica                  1
Burkina Faso              1
Albania                   1
Name: Country, Length: 227, dtype: int64

### Date fixing

Trying to cast the `date` colomn 

Keeping Years

In [8]:
data['date'] = data['date'].replace(['date unk.'],"01-Jan-1900")

In [9]:
data['date'] = data['date'].fillna("01-Jan-1900")

In [10]:
data['fatilites'] = data['fatilites'].fillna(0)

In [11]:
import datetime as dt

data['date'] = data['date'].apply(lambda x: x.strip())

def trim_date(x):
    if len(x) >8 : return x[3:]
    else: return x

data['date'] = data['date'].apply(lambda x: trim_date(x))

def datefiller(x):
    
    if x == '??-???-????': return '01-Jan-1900'
    elif x[:6] == '??-???': return '01-Jan'+x[6:]
    elif x[:2] == '??': return '01'+x[2:]
    else: return x


data['date'] = data['date'].apply(lambda x:datefiller(x))
data['year'] = data['date'].str[4:].str.lower()


In [12]:
data['year'] = data['year'].replace(['022'],"2022")

In [9]:
data['year'].value_counts()

1944    1307
1945    1117
1943     637
1969     360
1946     336
        ... 
1924       3
1922       3
1925       2
1919       2
1927       1
Name: year, Length: 105, dtype: int64

In [70]:
data['year'].nunique()

105

In [10]:
year_country=data.groupby(['year', 'Country']).size()

In [11]:
year_country

year  Country            
1900  Albania                 1
      American Samoa          1
      Andorra                 1
      Anguilla                1
      Antigua and Barbuda     1
                             ..
2022  Thailand                1
      UAE                     1
      USA                    31
      Ukraine                 8
      Venezuela               1
Length: 5708, dtype: int64

ploting

In [16]:
!pip install plotly_express

You should consider upgrading via the 'C:\Users\User\anaconda3\python.exe -m pip install --upgrade pip' command.


In [19]:
pip install plotly==5.10.0

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: plotly
    Found existing installation: plotly 5.4.0
    Uninstalling plotly-5.4.0:
      Successfully uninstalled plotly-5.4.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade nbformat

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [13]:
import plotly.express as px

In [24]:
fig = px.scatter(data, x="year", y="fatilites", color="Country")

In [19]:
fig.show()

New data frame
year - country - #rows (count(accidents)) - #fatilities(sum)

In [26]:
year_country=data.groupby(['year', 'Country']).size()

In [27]:
type(year_country)

pandas.core.series.Series

In [28]:
year_country = year_country.to_frame().reset_index()

In [30]:
year_country = year_country.rename(columns= {0: 'count_accidents'})
year_country.head()

,year,Country,count_accidents
0,1900,Albania,1
1,1900,American Samoa,1
2,1900,Andorra,1
3,1900,Anguilla,1
4,1900,Antigua and Barbuda,1


In [31]:
import plotly.express as px

fig = px.scatter(year_country, x="year", y="count_accidents", size="count_accidents", color="Country",
           hover_name="Country", log_x=True, size_max=60)
fig.show()

In [34]:
year_country.sort_values("count_accidents")

,year,Country,count_accidents
0,1900,Albania,1
3131,1983,Israel,1
3134,1983,Kazakhstan,1
3135,1983,Kyrgyzstan,1
3136,1983,Latvia,1
...,...,...,...
522,1944,India,144
611,1945,Myanmar,149
581,1945,China,163
483,1943,USA,171


In [ ]:
df2 = data.groupby(['year','Country'])['fatilites'].agg(['sum','count'])
print(df2)

In [39]:
data

,Country,date,Air-craft type,registration name/mark,operator,fatilites,location,category,year
0,Australia,Dec-2016,Boeing 737-8FE (WL),VH-VUP,Virgin Australia Airlines,0,Hobart Airpo...,A2,2016
1,Ukraine,Feb-1987,Antonov An-26,NaN,Soviet AF,9,near Stadnitsa,A1,1987
2,Iran,Feb-2003,Ilyushin Il-76MD,15-2280,Iranian Revolutionary Guard,275,near Kerman,A1,2003
3,Germany,May-1990,Transall C-160D,50+39,German AF,10,near Rodenbach,A1,1990
4,Russia,Jan-1943,Junkers Ju-52/3m,NaN,German AF,0,Ostrow,A1,1943
...,...,...,...,...,...,...,...,...,...
21737,UAE,Aug-2022,Cessna 208 Caravan,NaN,NaN,0,near Abu Dhabi-Ba...,A1,2022
21738,Russia,Sep-2022,Antonov An-2R,RA-56534,2nd Arkhangelsk United Aviation Division,0,Nes,A2,2022
21739,Poland,Sep-2022,Shorts SC.7 Skyvan 3M-400,SP-HIP,SkyForce,2,near Piotrków Try...,A1,2022
21740,Latvia,ep-2022,Cessna 551 Citation II/SP,OE-FGR,GG Rent,4,near Ventspils (B...,A1,2022


In [ ]:
data.groupby(['year','Country']).agg(MySum=('fatilites', 'sum'), MyCount=('Country', 'count'))

In [57]:
data.groupby('Country').size().sort_values()[-10:]

Country
Colombia           455
China              507
Germany            509
France             546
Brazil             606
India              696
Canada             816
United Kingdom     836
Russia            1410
USA               4303
dtype: int64

In [46]:
type(data["fatilites"][500])

str

In [ ]:
data['fatilites'].astype('int32').dtypes

In [48]:
data['fatilites']

0          0
1          9
2        275
3         10
4          0
        ... 
21737      0
21738      0
21739      2
21740      4
21741     10
Name: fatilites, Length: 21742, dtype: object

About Countries

In [59]:
countries = data.groupby('Country').size().sort_values(ascending=[False])
countries[:15]

Country
USA               4303
Russia            1410
United Kingdom     836
Canada             816
India              696
Brazil             606
France             546
Germany            509
China              507
Colombia           455
Indonesia          428
Italy              352
Myanmar            344
Mexico             342
Vietnam            320
dtype: int64

In [60]:
type(countries)

pandas.core.series.Series

In [74]:
countries_df=countries.to_frame().reset_index()

In [75]:
countries_df = countries_df.rename(columns= {0: 'count_accidents'})
countries_df.head()

,Country,count_accidents
0,USA,4303
1,Russia,1410
2,United Kingdom,836
3,Canada,816
4,India,696


In [68]:
100*countries_df['count_accidents'][:20].sum()/countries_df['count_accidents'].sum()

63.74298592585779

In [69]:
countries_df['count_accidents'][:20]

Country
USA                 4303
Russia              1410
United Kingdom       836
Canada               816
India                696
Brazil               606
France               546
Germany              509
China                507
Colombia             455
Indonesia            428
Italy                352
Myanmar              344
Mexico               342
Vietnam              320
Australia            313
Philippines          300
Papua New Guinea     270
Japan                259
Argentina            247
Name: count_accidents, dtype: int64

In [73]:
countries_df

,count_accidents
Country,
USA,4303
Russia,1410
United Kingdom,836
Canada,816
India,696
...,...
Dominica,1
East Timor,1
Equatorial Guinea,1


In [80]:
#df = px.data.gapminder()
fig = px.choropleth(countries_df, locations="Country", color="count_accidents", hover_name="Country", range_color=[1,4303])
fig.show()

In [83]:
fig = px.scatter_geo(year_country[:10], locations="Country", color="Country", hover_name="Country", size="count_accidents",
               animation_frame="year", projection="natural earth")
fig.show()

Each decade instead of country

In [85]:
year = data['year'].astype(int)
year = year.round(-1)
print(year)

0        2020
1        1990
2        2000
3        1990
4        1940
         ... 
21737    2020
21738    2020
21739    2020
21740    2020
21741    2020
Name: year, Length: 21742, dtype: int32


In [86]:
year = year.value_counts()
year  = pd.DataFrame(year.sort_index())
print(year)

      year
1900   104
1920    28
1930   147
1940  4041
1950  2316
1960  2220
1970  2523
1980  2477
1990  2147
2000  2433
2010  1817
2020  1489
